In [54]:
import serial # pyserial library
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import time

In [32]:
# Define the serial port and the baud rate
port = 'COM4'  # Replace with your serial port name, e.g., '/dev/ttyUSB0' on Linux
baud_rate = 9600  # Common baud rate

In [71]:
try:
    # Open the serial port
    ser = serial.Serial(port, baud_rate, timeout=1)
    
    # Function to send command to the serial port
    def send_command(change):
        joint = change['owner'].description.split(' ')[1].lower()  # Get joint identifier
        angle = change['new']
        command = f"{joint} {angle}\n\r"
        print(f"Message to be sent: {command.strip()}")
        ser.write(command.encode('ascii'))
        clear_output(wait=True)  # Clear previous output to keep it clean
        print(f"Sent command: {command.strip()}")
    
    # Function to save the current joint settings to a file
    def save_settings():
        with open('joint_settings.json', 'a') as file:
            for joint, value in sliders.items():
                command = f"{joint} {value.value}\n"
                file.write(command)
        print("Joint settings saved to joint_settings.json")
    
    # Function to execute the settings from the file
    def execute_saved_settings():
        try:
            with open('joint_settings.json', 'r') as file:
                for line in file:
                    joint, angle = line.strip().split()
                    command = f"{joint} {angle}\n\r"
                    ser.write(command.encode('ascii'))
                    sliders[joint].value = int(angle)  # Update slider to reflect current position
                    print(f"Executing command: {command.strip()}")
        except FileNotFoundError:
            print("No saved settings file found.")

    # Function to reset all joints to 90 degrees
    def home_position():
        for joint, slider in sliders.items():
            slider.value = 90
            command = f"{joint} 90\n\r"
            ser.write(command.encode('ascii'))
            print(f"Resetting {joint} to 90 degrees")
        print("All joints reset to home position (90 degrees).")

    # Function to transition a joint from a start point to an end point
    def transition_joint(joint, start, end, step=1, delay=0.05):
        if start < end:
            for angle in range(start, end + 1, step):
                command = f"{joint} {angle}\n\r"
                ser.write(command.encode('ascii'))
                sliders[joint].value = angle  # Update slider to reflect current position
                print(f"Transitioning {joint} to {angle} degrees")
                time.sleep(delay)
        else:
            for angle in range(start, end - 1, -step):
                command = f"{joint} {angle}\n\r"
                ser.write(command.encode('ascii'))
                sliders[joint].value = angle  # Update slider to reflect current position
                print(f"Transitioning {joint} to {angle} degrees")
                time.sleep(delay)
                
    # Function to get the current position of all sliders
    def get_current_positions():
        positions = {joint: slider.value for joint, slider in sliders.items()}
        print("Current joint positions:", positions)
        return positions            
    
    # Function to execute saved settings by transitioning joints
    def execute_saved_settings_with_transition():
        try:
            with open('joint_settings.json', 'r') as file:
                for line in file:
                    joint, target_angle = line.strip().split()
                    target_angle = int(target_angle)
                    current_positions = get_current_positions()
                    start_angle = current_positions[joint]
                    transition_joint(joint, start_angle, target_angle)
        except FileNotFoundError:
            print("No saved settings file found.")
    
    # Create sliders for each joint (a to f)
    sliders = {}
    slider_widgets = []
    for joint in ['a', 'b', 'c', 'd', 'e', 'f']:
        slider = widgets.IntSlider(value=90, min=0, max=180, step=1, description=f'Joint {joint.upper()}')
        slider.observe(send_command, names='value')
        sliders[joint] = slider
        slider_widgets.append(slider)
    sliders_box = widgets.VBox(slider_widgets)
    
    # Button to save the current joint settings
    save_button = widgets.Button(description="Save Current Settings")
    save_button.on_click(lambda x: save_settings())
    display(save_button)
    
    # Button to execute the saved settings
    execute_saved_button = widgets.Button(description="Execute Saved Settings")
    execute_saved_button.on_click(lambda x: execute_saved_settings())
    display(execute_saved_button)

    # Button to reset all joints to home position
    home_button = widgets.Button(description="Home Position")
    home_button.on_click(lambda x: home_position())
    display(home_button)

    joint_selector = widgets.Dropdown(options=['a', 'b', 'c', 'd', 'e', 'f'], description='Joint:')
    start_box = widgets.BoundedIntText(value=0, min=0, max=180, step=1, description='Start:')
    end_box = widgets.BoundedIntText(value=180, min=0, max=180, step=1, description='End:')
    move_button = widgets.Button(description="Move Joint")
    
    transition_box = widgets.HBox([joint_selector, start_box, end_box, move_button])
    
    # Button to execute saved settings with transition
    execute_transition_button = widgets.Button(description="Execute Saved Settings with Transition")
    execute_transition_button.on_click(lambda x: execute_saved_settings_with_transition())
    display(execute_transition_button)
    
    def on_move_button_click(_):
        joint = joint_selector.value
        start = start_box.value
        end = end_box.value
        transition_joint(joint, start, end)
    
    move_button.on_click(on_move_button_click)

    # Button to get current positions of sliders
    get_positions_button = widgets.Button(description="Get Current Positions")
    get_positions_button.on_click(lambda x: get_current_positions())
    
    close_button = widgets.Button(description="Close Serial Port")
    close_button.on_click(lambda x: close_serial_port())
    display(close_button)   

    # Arrange buttons in a structured layout
    buttons_box = widgets.VBox([
        widgets.HBox([save_button, execute_saved_button, execute_transition_button]),
        widgets.HBox([home_button, get_positions_button, close_button])
    ])
    
    # Display all widgets in a structured layout
    display(sliders_box, transition_box, buttons_box)
    
    # Close the serial port when done
    def close_serial_port():
        if ser.is_open:
            ser.close()
            print("Serial port closed.")
    
    # Create a button to close the serial port


except serial.SerialException as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Button(description='Save Current Settings', style=ButtonStyle())

Button(description='Execute Saved Settings', style=ButtonStyle())

Button(description='Home Position', style=ButtonStyle())

Button(description='Execute Saved Settings with Transition', style=ButtonStyle())

Button(description='Close Serial Port', style=ButtonStyle())